Trabalho de Confiabilidade de Sistemas Elétricos de Potência

Alunos: 

Gabriel Halfeld

Vinícius Albuquerque Cabral

Data de entrega: 17/12/2024

In [1]:
# =============================================
# Importação de pacotes Python
# =============================================
import time
import numpy as np
import pyomo as pyo
import pandas as pd

# =============================================
# Importação de pacotes implementados
# =============================================

# Faz a leitura dos dados do sistema armazenado em csv
from data_processing import load_all_data

# Faz o cálculo do fluxo de potência ótimo considerando a disponibilidade de geração ou de linha
from calcula_fpo import calcular_fpo

Leitura dos dados de entrada

In [3]:
data = load_all_data()
D_GEN = data['D_GEN']
D_LIN = data['D_LIN']
D_LOAD = data['D_LOAD']

# Número de geradores
nger = len(D_GEN)

# Número de linhas
nlinhas = len(D_LIN)

Definição dos índices através da enumeração de estados

Como são avaliados apenas no critério de contingência N-1, serão considerados dois casos:

    -> todas as possibilidades em que há perda de uma única unidade geradora e todas as linhas são mantidas e 
    -> todas as possibilidades em que as unidades geradoras são mantidas e uma única linha é desligada

In [4]:
# Número de geradores
GERADORES_ON = np.ones((nger,))

# Número de linhas
LINHAS_ON = np.ones((nlinhas,))

# Os estados possíveis envolvem a remoção de um único elemento: um gerador ou uma linha. O estado do sistema 
# funcionando com todos os elementos ativos também deve ser considerado. Por isso, há o +1.
nestados = nger + nlinhas + 1

In [5]:
# Colunas: Probabilidade | Corte de carga | Tempo de reparo do estado | Taxa de transição de Estado de volta ao 1 
# | Taxa de transição de Estado saindo do 1
MATRIZ_ESTADOS = np.zeros((nestados,5))

MATRIZ_A = np.zeros((nestados,nestados))

# O primeiro estado é o estado operativo de todas as unidades e linhas
MATRIZ_ESTADOS[0,1] = calcular_fpo(GERADORES_ON, LINHAS_ON, False)

for iest in range(nestados-1):

    if iest < nger:

        ESTADO_GER = GERADORES_ON.copy()
        ESTADO_GER[iest] = 0

        MATRIZ_ESTADOS[iest+1,1] = calcular_fpo(ESTADO_GER, LINHAS_ON, False)
        MATRIZ_ESTADOS[iest+1,2] = D_GEN['Tempo de reparo(h)'][iest]

        lambda_falha = D_GEN['Falhas/ano'][iest]
        mu_reparo = 8760/D_GEN['Tempo de reparo(h)'][iest]
        
        MATRIZ_ESTADOS[iest+1,3] = mu_reparo
        MATRIZ_ESTADOS[iest+1,4] = lambda_falha
        
        MATRIZ_ESTADOS[0,4] += lambda_falha
        
        MATRIZ_A[0,iest+1] += lambda_falha
        MATRIZ_A[iest+1,0] += mu_reparo

        MATRIZ_A[0,0] += -lambda_falha
        MATRIZ_A[iest+1,iest+1] += -mu_reparo

    else:
        
        indice_linha = iest - nger

        ESTADO_LIN = LINHAS_ON.copy()
        ESTADO_LIN[indice_linha] = 0

        MATRIZ_ESTADOS[iest+1,1] = calcular_fpo(GERADORES_ON, ESTADO_LIN, False)
        MATRIZ_ESTADOS[iest+1,2] = D_LIN['Tempo de Reparo'][indice_linha]

        lambda_falha = D_LIN['Falhas/(ano.milha)'][indice_linha]*D_LIN['Comprimento'][indice_linha]
        mu_reparo = 8760/D_LIN['Tempo de Reparo'][indice_linha]

        MATRIZ_ESTADOS[iest+1,3] = mu_reparo
        MATRIZ_ESTADOS[iest+1,4] = lambda_falha
        MATRIZ_ESTADOS[0,4] += lambda_falha

        MATRIZ_A[0,iest+1] += lambda_falha
        MATRIZ_A[iest+1,0] += mu_reparo

        MATRIZ_A[0,0] += -lambda_falha
        MATRIZ_A[iest+1,iest+1] += -mu_reparo

MATRIZ_A[:,-1] = 1        

VETOR_B = np.zeros((1,nestados))
VETOR_B[0,-1] = 1

PROB = np.linalg.matmul(VETOR_B, np.linalg.inv(MATRIZ_A))

MATRIZ_ESTADOS[:,0] = PROB[0,:]

MATRIZ_ESTADOS = pd.DataFrame(MATRIZ_ESTADOS,columns=['Probabilidade','Corte de Carga','Tempo de Reparo [h]','mu_reparo','lambda_falha'])
#MATRIZ_COPT.style.hide()

display(MATRIZ_ESTADOS)


,Probabilidade,Corte de Carga,Tempo de Reparo [h],mu_reparo,lambda_falha
0,0.536179,0.000000,0.0,0.0,141.854
1,0.059586,0.000000,50.0,175.2,19.470
2,0.059586,0.000000,50.0,175.2,19.470
3,0.059586,0.000000,50.0,175.2,19.470
4,0.010944,6.250000,40.0,219.0,4.470
5,0.022341,0.000000,50.0,175.2,7.300
6,0.022341,0.000000,50.0,175.2,7.300
7,0.022341,0.000000,50.0,175.2,7.300
8,0.022341,0.000000,50.0,175.2,7.300
9,0.022341,0.000000,50.0,175.2,7.300


In [6]:
# TESTE DA ACUMULADA
# Neste teste, busca-se avaliar se o somatório dos produtos da probabilidade pelas taxas de reparo dos estados de falha é equivalente
# à probabilidade do estado operativo nominal multiplicada pela taxa de falha do estado nominal. Para avaliar, a variável teste1 deve ser zero.
PARTE1 = np.sum(MATRIZ_ESTADOS['Probabilidade']*MATRIZ_ESTADOS['mu_reparo'])
PARTE2 = MATRIZ_ESTADOS['Probabilidade'][0]*MATRIZ_ESTADOS['lambda_falha'][0]
teste1 = PARTE1 - PARTE2

print('teste1:', teste1)

teste1: 0.0


In [7]:
# TESTE DO TEOREMA DA PROBABILIDADE
# Neste teste, busca-se avaliar se o somatório das probabilidades dos estados operativos é igual a 1.
teste2 = np.sum(MATRIZ_ESTADOS['Probabilidade'])
print('teste2:', teste2)

teste2: 1.0


In [9]:
# Cálculo do LOLP do sistema
LOLP = np.sum(MATRIZ_ESTADOS['Probabilidade'][MATRIZ_ESTADOS['Corte de Carga']>0])
print('LOLP - LOSS OF LOAD PROBABILITY')
print('Probabilidade de ocorrência de perda de carga')
print('O LOLP é calculado através da soma das probabilidades dos estados em que há corte de carga:')
print('LOLP:',np.round(LOLP*100,2),'%')
print('----------------------------------------------')

# Cálculo do LOLE do sistema
LOLE = LOLP*8760
print('LOLE - LOSS OF LOAD EXPECTATION')
print('É o tempo médio esperado em que não haverá atendimento da carga do sistema.')
print('O LOLE é calculado através do produto do LOLP por 8760:')
print('LOLE:',np.round(LOLP*8760,2),'h')
print('----------------------------------------------')

# Cálculo do LOLF do sistema
LOLF = np.sum( MATRIZ_ESTADOS['Probabilidade'][MATRIZ_ESTADOS['Corte de Carga']>0] * MATRIZ_ESTADOS['mu_reparo'][MATRIZ_ESTADOS['Corte de Carga']>0])
print('LOLF - LOSS OF LOAD FREQUENCY')
print('É a frequência média de perda de carga definida como falhas/período')
print('O LOLF é calculado através da soma dos produtos da probabilidade dos estados em que há perda de carga pela taxa de reparo da falha do estado:')
print('LOLF:',np.round(LOLF,2),'falhas/ano')
print('----------------------------------------------')

# Cálculo do LOLD do sistema
LOLD = LOLP/LOLF*8760
print('LOLD - LOSS OF LOAD DURATION')
print('Duração média da perda de carga.')
print('Pela definição, o LOLD é a razão entre o LOLP e o LOLF.')
print('LOLD:',np.round(LOLD,2),'h')
print('----------------------------------------------')

# Cálculo do EPNS do sistema
EPNS = np.sum( MATRIZ_ESTADOS['Probabilidade'][MATRIZ_ESTADOS['Corte de Carga']>0] * MATRIZ_ESTADOS['Corte de Carga'][MATRIZ_ESTADOS['Corte de Carga']>0])
print('EPNS - EXPECTED POWER NOT SUPPLIED')
print('Valor experado da carga não atendida, em MW.')
print('EPNS:',np.round(EPNS,2),'MW')
print('----------------------------------------------')

# Cálculo do EENS do sistema
EENS = EPNS*8760
print('EENS - EXPECTED ENERGY NOT SUPPLIED')
print('Valor experado da energia não atendida, em MWh.')
print('EENS:',np.round(EENS,2),'MWh')
print('----------------------------------------------')


LOLP - LOSS OF LOAD PROBABILITY
Probabilidade de ocorrência de perda de carga
O LOLP é calculado através da soma das probabilidades dos estados em que há corte de carga:
LOLP: 2.05 %
----------------------------------------------
LOLE - LOSS OF LOAD EXPECTATION
É o tempo médio esperado em que não haverá atendimento da carga do sistema.
O LOLE é calculado através da soma dos produtos da probabilidade dos estados em que há perda de carga pelo tempo de reparo da falha do estado:
LOLE: 179.62 h
----------------------------------------------
LOLF - LOSS OF LOAD FREQUENCY
É a frequência média de perda de carga definida como falhas/período
O LOLF é calculado através da soma dos produtos da probabilidade dos estados em que há perda de carga pela taxa de reparo da falha do estado:
LOLF: 10.77 falhas/ano
----------------------------------------------
LOLD - LOSS OF LOAD DURATION
Duração média da perda de carga.
Pela definição, o LOLD é a razão entre o LOLP e o LOLF.
LOLD: 16.67 h
---------------